In [43]:
import numpy as np
import pysindy as ps
import matplotlib.pyplot as plt
import sympy as sp
from scipy.integrate import solve_ivp
from ekf_vindy.plotting import plotter
from ekf_vindy.utils import add_noise_with_snr
from ekf_vindy.filters.config import DynamicsConfig
from ekf_vindy.filters.ekf import EKF
from scipy.integrate import odeint

seed = 29
np.random.seed(seed)

# EKF-SINDy with Duffing oscillator

We examine a simulation with a Duffing oscillator, and we want to inform the filter about energy conservation. Some possible examples

- Start from a wrong model (obtained by SINDy), where there's a damping term. Such a damping term must fade during online inference. However, we should be able to see the term disappear faster in the physics-informed EKF case.
- The other way around, learn a system where energy is conserved, and energy gradually is lost via a damping term that becomes active after a certain point during the online phase. Again, we should see the EKF be faster in adapting to the environment given the constraint. 

## Duffing oscillator

The undamped Duffing oscillator is the following

$$
\ddot{x}+\alpha x + \beta x^3 + \gamma x^5= 0
$$

Energy is conserved (a physical constraint in an isolated system). We will also considered the damped oscillator.

$$
\ddot{x}+\delta \dot{x}+\alpha x+ \beta x^3  + \gamma x^5= 0, \,\,\delta > 0
$$

We consider the first-order version of the damped oscillator

$$
\begin{aligned}
\dot{x}_0 &= x_1,\\
\dot{x}_1 &= - \alpha x_0-  \beta x_0^3 -\gamma x_0^5 - \delta x_1
\end{aligned}
$$

In [61]:
def duffing_oscillator(x, t):
    """
    Duffing oscillator ODE
    """
    alpha = 1   # Linear stiffness
    beta = 1     # Nonlinear stiffness
    gamma = 1    # Higher-order stiffness
    delta = 0.5  # Damping coefficient
    
    dxdt = x[1]
    dvdt = - alpha * x[0] - beta * x[0] ** 3 - gamma * x[0] ** 5 - delta * x[1]
    
    return [dxdt, dvdt]

The total energy of this system is given by the Hamiltonian (potential and kinetic energy). 

$$
H(x(t), \dot{x}(t)) = \frac{1}{2}\dot{x}^2(t) +\frac{1}{2}\alpha x^2(t) +\frac{1}{4}\beta x^4(t) + \frac{1}{6}\gamma x^6(t). 
$$

So want we want to enforce is that the energy at the initial state (assuming there's no loss), is equal to the energy in successive states. So

$$
g(x(t), \dot{x}(t)) = H(x(t), \dot{x}(t)) - H(x(0), \dot{x}(0)) = 0
$$

Since we will put this into the filter, as a pseudo-observation, we must linearize the constraint $g(x(t), \dot{x}(t))$, so we need its Jacobian. In this case, we can easily derive in closed-form.

In [62]:
def energy(x):
    """
    Hamiltonian (total energy) of the Duffing oscillator
    """
    alpha = 1   # Linear stiffness
    beta = 1     # Nonlinear stiffness
    gamma = 1    # Quintic term coefficient
    
    kinetic = 0.5 * x[1]**2
    potential = 0.5 * alpha * x[0]**2 + 0.25 * beta * x[0]**4 + (1/6) * gamma * x[0]**6
    
    return kinetic + potential

def jacobian_constraint(x):
    """
    Jacobian of the energy constraint
    """
    alpha = 1   # Linear stiffness
    beta = 1     # Nonlinear stiffness
    gamma = 1    # Quintic term coefficient
    
    dH_dx = np.array([alpha * x[0] + beta * x[0]**3 + gamma * x[0]**5, x[1]])
    
    return dH_dx

Generate trajectories in the undamped case and train SINDy...

In [63]:
# we select points around (1,0) in phase space

n_train = 20
std = 1.0
mean_ic = np.array([1.0, 0.0])
x_0 = np.random.randn(n_train, 2) * std
time_instances = np.arange(0, 50, 0.1)
x_train = []


for i in range(n_train):
    sol = odeint(duffing_oscillator, x_0[i, :], time_instances)
    x_train.append(sol)

x_train = np.array(x_train)
x_train = [noisy_traj for noisy_traj in add_noise_with_snr(x_train, snr=15)]

model = ps.SINDy(feature_names=['x0', 'x1'],
                 feature_library=ps.PolynomialLibrary(degree=3),
                 optimizer=ps.STLSQ(threshold=5e-2))

model.fit(x_train, t=0.1, multiple_trajectories=True)
model.print()

(x0)' = 0.912 x1 + 0.071 x0^2 + -0.071 x0^3
(x1)' = -0.061 1 + 2.662 x0 + -0.493 x1 + 1.451 x0^2 + -0.113 x0 x1 + -6.896 x0^3 + 0.091 x0^2 x1 + -0.054 x0 x1^2


What if system suddenly becomes conservative? So the damping term $\delta$ goes to zero abruptly. We define the constraint object and set a very small noise